# Experimentation and Uplift Testing

Trial stores were performed in stores 77, 86 and 88

In [1]:
import re
import xlrd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib.patches as mpatches
from datetime import datetime, timedelta
from scipy.stats import ttest_ind, linregress

pd.options.mode.chained_assignment = None  

In [11]:
customer_data = pd.read_csv('QVI_purchase_behaviour.csv')
transaction_data = pd.read_excel('QVI_transaction_data.xlsx')

#cleaned transaction_data merged with customer_data
data = pd.read_csv('QVI_data.csv', parse_dates=['DATE']) 

## I. Data Cleaning

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int64         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int64         
 3   TXN_ID            264834 non-null  int64         
 4   PROD_NBR          264834 non-null  int64         
 5   PROD_NAME         264834 non-null  object        
 6   PROD_QTY          264834 non-null  int64         
 7   TOT_SALES         264834 non-null  float64       
 8   PACK_SIZE         264834 non-null  int64         
 9   BRAND             264834 non-null  object        
 10  LIFESTAGE         264834 non-null  object        
 11  PREMIUM_CUSTOMER  264834 non-null  object        
 12  MONTH             264834 non-null  object        
 13  YEAR              264834 non-null  int64         
dtypes: d

In [17]:
data['MONTH'] = data.DATE.dt.month_name()
data['YEAR'] = data.DATE.dt.year

In [18]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,MONTH,YEAR
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,October,2018
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,September,2018
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,March,2019
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,March,2019
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,November,2018


In [23]:
data[data.STORE_NBR.isin([77,86,88])]['MONTH'].nunique()

12

In [24]:
data[(data.STORE_NBR.isin([77,86,88])) & (data.DATE < "2019-02-01")]['MONTH'].nunique()

7

There are 3 trials which began in Feb 2019. In order to compare with a control group, we would need to have a control group whose pre-trial dates adn measures are the most similar to our trial stores.

In [34]:
txn_per_store_month = data.groupby(['STORE_NBR','MONTH'])['TXN_ID'].count().reset_index().groupby('STORE_NBR')['MONTH'].count().reset_index()
possible_cntrl_stores = txn_per_store_month[(txn_per_store_month.MONTH == 12) & ~(txn_per_store_month.STORE_NBR.isin([77,86,88]))].reset_index(drop=True)

Now, we need to define the measures for our possible cont